#MIS 6346.502 - Big Data - F23
##Project 2
##Group 8

##2A

In [1]:
# Load the data from HDFS
data_path = "/project_execute/blockchain_block_data.csv"  # Replace with the actual HDFS path to your data
raw_data = sc.textFile(data_path)

# Skip the header row
header = raw_data.first()
parsed_data = raw_data.filter(lambda line: line != header).map(lambda line: line.split(','))

# Filter out rows with insufficient elements
parsed_data = parsed_data.filter(lambda x: len(x) >= 14 and x[13].isdigit() and x[9].isdigit()).map(lambda x: (int(x[13]), x))

In [2]:
# 1. How many total blocks are there in your dataset?
total_blocks = parsed_data.count()
print("1. Total Blocks:", total_blocks)

('1. Total Blocks:', 1016)


In [3]:
# 2. What is the largest block height among the blocks in your dataset?
largest_block_height = parsed_data.max(lambda x: x[0])[0]
print("2. Largest Block Height:", largest_block_height)

('2. Largest Block Height:', 813369)


In [4]:
# 3. What is the date and time for that block?
largest_block_info = parsed_data.filter(lambda x: x[0] == largest_block_height).collect()

print("3. Largest Block Info:", largest_block_info)

if largest_block_info:
    date_time = largest_block_info[0][1][4]  # Correct index to access date and time
    print("3A. Date and Time for the Largest Block:", date_time)
else:
    print("No information found for the largest block.")

('3. Largest Block Info:', [(813369, [u'00000000000000000000d8b8b6b8290b53b9abe897389b2e249d633d9e61386b', u'583360512', u'00000000000000000003c5b1e8534fbce432d8721d61120b0e0229033b8370ca', u'1638c939ca26253939161a41d8cfd87a83181a2bd4af25801288fec6bb416db8', u'2023-10-22 17:56:02', u'386178217', u"['00000000000000000002fcfe75d1d77960bd7474cd2d4c2365757fd49b76da4d']", u'4713973', u'1818101364', u'1337', u'1874504', u'813369', u'True', u'813369', u'3993380'])])
('3A. Date and Time for the Largest Block:', u'2023-10-22 17:56:02')


In [5]:
# 4. What is the highest number of transactions in your blocks?
filtered_data = parsed_data.filter(lambda x: len(x[1]) > 9)  # Filter out rows with insufficient elements
max_transactions = filtered_data.max(lambda x: int(x[1][9]))[1][9]
print("4. Highest Number of Transactions:", max_transactions)

('4. Highest Number of Transactions:', u'4158')


##2B

In [6]:
# Load the data from HDFS
data_path = "/project_execute/output_data.csv"  # Replace with the actual HDFS path to your data
raw_data = sc.textFile(data_path)

# Skip the header row
header = raw_data.first()
rdd = raw_data.filter(lambda line: line != header).map(lambda line: line.split(','))

# Filter out rows with insufficient elements
rdd = rdd.filter(lambda x: len(x) == 7 and x[6].isdigit()).map(lambda x: (int(x[6]), x))

In [7]:
# 1. Count the number of records
num_records = rdd.count()
print("Number of records: {}".format(num_records))

Number of records: 5048


In [8]:
# 2. Count the number of different days
num_unique_days = rdd.filter(lambda line: len(line[1]) > 1).map(lambda line: line[1][1].split()[0]).distinct().count()
print("Number of different days:", num_unique_days)

('Number of different days:', 1251)


In [9]:
# 3. Count records per each day
records_per_day = rdd.filter(lambda line: len(line[1]) > 1).map(lambda line: (line[1][1].split()[0], 1)).reduceByKey(lambda a, b: a + b)
print("Records per each day:")
records_per_day.collect()

Records per each day:


[(u'2021-10-01', 4),
 (u'2021-11-19', 4),
 (u'2007-10-26', 3),
 (u'2008-11-28', 3),
 (u'2020-12-31', 4),
 (u'2019-06-07', 4),
 (u'2021-11-12', 4),
 (u'2003-06-27', 3),
 (u'2020-01-24', 4),
 (u'2013-04-19', 4),
 (u'2008-11-14', 3),
 (u'2022-09-16', 4),
 (u'2006-08-11', 3),
 (u'2018-03-09', 4),
 (u'2021-03-26', 4),
 (u'2000-04-20', 3),
 (u'2015-11-13', 4),
 (u'2013-04-12', 4),
 (u'2002-06-14', 3),
 (u'2013-09-06', 4),
 (u'2014-09-19', 4),
 (u'2004-05-21', 3),
 (u'2001-08-31', 3),
 (u'2018-07-20', 4),
 (u'2013-12-20', 4),
 (u'2013-12-27', 4),
 (u'2018-07-27', 4),
 (u'2014-09-12', 4),
 (u'2015-09-11', 4),
 (u'2017-01-06', 4),
 (u'2003-06-20', 3),
 (u'2015-09-18', 4),
 (u'2007-06-08', 3),
 (u'2004-01-30', 3),
 (u'2019-09-27', 4),
 (u'2009-03-13', 3),
 (u'2010-02-26', 3),
 (u'2001-01-12', 3),
 (u'2001-03-16', 3),
 (u'2018-10-26', 4),
 (u'2010-07-02', 3),
 (u'2003-09-12', 3),
 (u'2011-03-25', 4),
 (u'2002-09-13', 3),
 (u'2021-01-15', 4),
 (u'2010-11-12', 4),
 (u'2017-11-17', 4),
 (u'2019-08-1

In [10]:
# 4. Extract symbols from the table
symbols = rdd.map(lambda line: line[1][0]).distinct()
print("Symbols in the table:")
symbols.collect()

Symbols in the table:


[u'AAPL', u'TSLA', u'IBM', u'MSFT', u'GOOGL']

In [11]:
# 5. Find the highest price for each symbol
highest_price_per_symbol = rdd.map(lambda line: (line[1][0], float(line[1][3]))).reduceByKey(max)
print("Highest price for each symbol:")
highest_price_per_symbol.collect()

Highest price for each symbol:


[(u'AAPL', 705.07),
 (u'TSLA', 2318.49),
 (u'IBM', 215.9),
 (u'MSFT', 366.78),
 (u'GOOGL', 122.28)]

In [12]:
# 6. Find the lowest price for each symbol
lowest_price_per_symbol = rdd.map(lambda line: (line[1][0], float(line[1][3]))).reduceByKey(min)
print("Lowest price for each symbol:")
lowest_price_per_symbol.collect()

Lowest price for each symbol:


[(u'AAPL', 13.63),
 (u'TSLA', 19.59),
 (u'IBM', 62.0),
 (u'MSFT', 16.4),
 (u'GOOGL', 122.1)]

In [13]:
# 7. Find the average price for each symbol
average_price_per_symbol = rdd.map(lambda line: (line[1][0], (float(line[1][2]) + float(line[1][3])) / 2)).groupByKey().mapValues(lambda values: sum(values) / len(values))
print("Average price for each symbol:")
average_price_per_symbol.collect()

Average price for each symbol:


[(u'AAPL', 174.82728649148794),
 (u'TSLA', 302.7974605031446),
 (u'IBM', 132.55394390075813),
 (u'MSFT', 100.49905610658772),
 (u'GOOGL', 122.21754999999996)]

In [14]:
# 8. Find the range of price for each symbol
range_price_per_symbol = rdd.map(lambda line: (line[1][0], float(line[1][3]) - float(line[1][4]))).reduceByKey(max)
print("Range of price for each symbol:")
range_price_per_symbol.collect()

Range of price for each symbol:


[(u'AAPL', 79.99000000000001),
 (u'TSLA', 623.47),
 (u'IBM', 24.069999999999993),
 (u'MSFT', 37.599999999999994),
 (u'GOOGL', 0.09999999999999432)]

In [15]:
# 9. Find the date on which each symbol experienced the highest price
date_highest_price_per_symbol = rdd.map(lambda line: (line[1][0], (line[1][1].split()[0], float(line[1][3])))).reduceByKey(lambda a, b: a if a[1] > b[1] else b)
print("Date on which each symbol experienced the highest price:")
date_highest_price_per_symbol.collect()

Date on which each symbol experienced the highest price:


[(u'AAPL', (u'2012-09-21', 705.07)),
 (u'TSLA', (u'2020-08-28', 2318.49)),
 (u'IBM', (u'2013-03-15', 215.9)),
 (u'MSFT', (u'2023-07-21', 366.78)),
 (u'GOOGL', (u'2023-10-27', 122.28))]

##2C

In [16]:
# Load the data from HDFS
blocks_data = "/project_execute/data/3/blocks_2023_Sep_10_to_15.csv"
blocks_raw_data = sc.textFile(blocks_data)

blocks_info_data = "/project_execute/data/3/blocks_info_2023_Sep_10_to_15.csv"
blocks_info_raw_data = sc.textFile(blocks_info_data)

tx_info_data = "/project_execute/data/3/tx_info_2023_Sep_10_to_15.csv"
tx_info_raw_data = sc.textFile(tx_info_data)

In [17]:
# Parse the data into a format that can be used for analysis
# Assuming the data is a CSV format as provided in your example
blocks_parsed_data = blocks_raw_data.map(lambda line: line.split(','))

blocks_info_parsed_data = blocks_info_raw_data.map(lambda line: line.split(','))

tx_info_parsed_data = tx_info_raw_data.map(lambda line: line.split(','))

In [18]:
# Convert block height to integer for comparison, skipping the header
header_blocks = blocks_parsed_data.first()
blocks_parsed_data = blocks_parsed_data.filter(lambda x: x != header_blocks).map(lambda x: (int(x[3]), x))

header_blocks_info = blocks_info_parsed_data.first()
blocks_info_parsed_data = blocks_info_parsed_data.filter(lambda x: x != header_blocks_info).map(lambda x: (int(x[9]), x))

header_tx_info = tx_info_parsed_data.first()
tx_info_parsed_data = tx_info_parsed_data.filter(lambda x: x != header_tx_info).map(lambda x: (int(x[15]), x))

In [19]:
# 1. How many total blocks are there in your blocks table?
total_blocks = blocks_parsed_data.count()
print("Total Blocks:", total_blocks)

('Total Blocks:', 920)


In [20]:
# 2. What is the largest block height among the blocks in your blocks table?
largest_block_height = blocks_parsed_data.max(key=lambda x: x[0])[0]
print("Largest Block Height:", largest_block_height)

('Largest Block Height:', 807751)


In [21]:
# 3. What is the date and time for that block?
largest_block_info = blocks_parsed_data.filter(lambda x: x[0] == largest_block_height).collect()[0]
date_time = largest_block_info[1][2]
print("Date and Time for the Largest Block:", date_time)

('Date and Time for the Largest Block:', u'2023-09-15 00:00:00')


In [22]:
# 4. What is the largest number of transactions in your blocks?
# Join blocks_info_parsed_data with tx_info_parsed_data on block height
joined_data = blocks_info_parsed_data.join(tx_info_parsed_data)

# Find the record with the largest number of transactions
largest_transaction_info = joined_data.max(key=lambda x: int(x[1][0][9]))
largest_num_transactions = largest_transaction_info[1][0][9]
print("Largest Number of Transactions in a Block:", largest_num_transactions)

('Largest Number of Transactions in a Block:', u'807290')


#3A

In [23]:
from pyspark import SparkContext
from pyspark.sql import Row, SQLContext

# Create an SQLContext from the SparkContext
sqlContext = SQLContext(sc)

# Load the data from HDFS
blocks_data = "/project_execute/blockchain_block_data.csv"
blocks_rdd = sc.textFile(blocks_data)
blocks_header = blocks_rdd.first()
blocks_rdd = blocks_rdd.filter(lambda line: line != blocks_header)
blocks_rows = blocks_rdd.map(lambda line: line.split(",")).map(lambda row: Row(*row))
blocks_df = sqlContext.createDataFrame(blocks_rows)

# Register the DataFrames as temporary tables to perform SQL queries
blocks_df.registerTempTable("blocks")

# Verify the data in dataframes
print("blocks")
result = sqlContext.sql("SELECT * FROM blocks limit 10")
result.show()

blocks
_1                   _2         _3                   _4                   _5                  _6        _7                   _8       _9         _10  _11     _12    _13  _14    _15    
00000000000000000... 973078528  00000000000000000... b479902ec7d9fa8cd... 2023-10-16 18:22:58 386178217 ['000000000000000... 29430531 3200379572 1658 1745738 812489 True 812489 3993545
00000000000000000... 592248832  00000000000000000... 0c51341821c78d61f... 2023-10-16 18:00:48 386178217 ['000000000000000... 21743284 4169245867 3818 1574438 812488 True 812488 3993062
00000000000000000... 587202560  00000000000000000... 6a1259d3abb42defa... 2023-10-16 17:44:12 386178217 ['000000000000000... 22114595 2451541900 2879 1856409 812487 True 812487 3993168
00000000000000000... 536928256  00000000000000000... ac6cecb0bfc196dbd... 2023-10-16 17:31:36 386178217 ['000000000000000... 27037245 962608425  3246 1522551 812486 True 812486 3993393
00000000000000000... 694657024  00000000000000000... 478638f991dd774

In [24]:
# 1. How many total blocks are there in your dataset?
total_blocks = sqlContext.sql("SELECT COUNT(*) as total_blocks FROM blocks").first().total_blocks
print("Total Blocks: {}".format(total_blocks))

Total Blocks: 1017


In [25]:
# 2. What is the largest block height among the blocks in your dataset?
max_block_height = sqlContext.sql("SELECT MAX(_3) as max_block_height FROM blocks").first().max_block_height
print("Largest Block Height: {}".format(max_block_height))

Largest Block Height: 00000000000000000004db4d7ecf76aeee94ad0dc6f6b65167c1ad8334e8fe58


In [26]:
# 3. What is the date and time for that block?
# Assuming your date and time information is stored in a column named 'timestamp'
block_info = sqlContext.sql("SELECT _5 FROM blocks WHERE _3 = '{}'".format(max_block_height)).first()
timestamp_value = block_info._5
print("Date and Time for Largest Block Height ({}): {}".format(max_block_height, timestamp_value))

Date and Time for Largest Block Height (00000000000000000004db4d7ecf76aeee94ad0dc6f6b65167c1ad8334e8fe58): 2023-10-15 21:03:11


In [27]:
# 4. What is the highest number of transactions in your blocks?
max_transactions = sqlContext.sql("SELECT MAX(_10) as max_transactions FROM blocks").first().max_transactions
print("Highest Number of Transactions: {}".format(max_transactions))

Highest Number of Transactions: 989


##3B

In [28]:
from pyspark import SparkContext
from pyspark.sql import Row, SQLContext

# Create an SQLContext from the SparkContext
sqlContext = SQLContext(sc)

# Load the data from HDFS
stocks_data = "/project_execute/output_data.csv"
stocks_rdd = sc.textFile(stocks_data)
stocks_header = stocks_rdd.first()
stocks_rdd = stocks_rdd.filter(lambda line: line != stocks_header)
stocks_rows = stocks_rdd.map(lambda line: line.split(",")).map(lambda row: Row(*row))
stocks_df = sqlContext.createDataFrame(stocks_rows)

# Register the DataFrames as temporary tables to perform SQL queries
stocks_df.registerTempTable("stocks_table")

# Verify the data in dataframes
print("stocks_table")
result = sqlContext.sql("SELECT * FROM stocks_table limit 10")
result.show()

stocks_table
_1  _2                  _3       _4       _5       _6       _7    
IBM 2023-10-27 19:37:00 142.3000 142.3000 142.3000 142.3000 2     
IBM 2023-10-27 19:36:00 142.3000 142.3000 142.3000 142.3000 10    
IBM 2023-10-27 19:12:00 142.4100 142.4100 142.4100 142.4100 10    
IBM 2023-10-27 19:06:00 142.5500 142.6900 142.5500 142.6900 101   
IBM 2023-10-27 19:05:00 142.5400 142.5500 142.5400 142.5500 100   
IBM 2023-10-27 19:04:00 142.4600 142.4600 142.3800 142.3800 10    
IBM 2023-10-27 19:02:00 142.5500 142.5500 142.5500 142.5500 20    
IBM 2023-10-27 19:00:00 142.5200 142.5200 142.5200 142.5200 553804
IBM 2023-10-27 18:57:00 142.4800 142.4800 142.4800 142.4800 1     
IBM 2023-10-27 18:52:00 142.5500 142.5500 142.5500 142.5500 2     


In [29]:
# 1. How many records are there in the table?
record_count = sqlContext.sql("SELECT COUNT(*) as record_count FROM stocks_table").first().record_count
print("Number of Records: {}".format(record_count))

Number of Records: 5048


In [30]:
# 2. How many different days are there in the table?
distinct_days = sqlContext.sql("SELECT COUNT(DISTINCT _2) as distinct_days FROM stocks_table").first().distinct_days
print("Number of Different Days: {}".format(distinct_days))

Number of Different Days: 1442


In [31]:
# 3. How many records per each day are there in the table?
records_per_day = sqlContext.sql("SELECT _2 as day, COUNT(*) as records_per_day FROM stocks_table GROUP BY _2")
records_per_day.show()

day                 records_per_day
2004-04-23          3              
2021-07-23          4              
2006-03-31          3              
2010-05-21          3              
2022-09-02          4              
2016-04-22          4              
2004-07-30          3              
2011-07-01          4              
2005-06-03          3              
2013-11-22          4              
2001-11-23          3              
1999-12-23          3              
2017-06-02          4              
2023-10-27 18:35:00 6              
2019-05-10          4              
2013-07-19          4              
2023-10-27 15:42:00 2              
2008-04-18          3              
2020-01-31          4              
2002-05-17          3              


In [32]:
# 4. What are the symbols in the table?
symbols = sqlContext.sql("SELECT DISTINCT _1 as symbol FROM stocks_table")
symbols.show()

symbol
AAPL  
GOOGL 
IBM   
MSFT  
TSLA  


In [33]:
# 5. What is the highest price for each symbol?
highest_price = sqlContext.sql("SELECT _1 as symbol, MAX(_4) as highest_price FROM stocks_table GROUP BY _1")
highest_price.show()

symbol highest_price
AAPL   99.7500      
GOOGL  122.2800     
IBM    99.8600      
MSFT   98.9800      
TSLA   987.6900     


In [34]:
# 6. What is the lowest price for each symbol?
lowest_price = sqlContext.sql("SELECT _1 as symbol, MIN(_4) as lowest_price FROM stocks_table GROUP BY _1")
lowest_price.show()

symbol lowest_price
AAPL   100.0000    
GOOGL  122.1000    
IBM    100.0000    
MSFT   100.1100    
TSLA   1005.0000   


In [35]:
# 7. What is the average price for each symbol?
average_price = sqlContext.sql("SELECT _1 as symbol, AVG(_4) as average_price FROM stocks_table GROUP BY _1")
average_price.show()

symbol average_price     
AAPL   177.38589533678785
GOOGL  122.22504999999994
IBM    133.81555396278438
MSFT   101.57014389341236
TSLA   311.41679962264146


In [36]:
# 8. What is the range of price for each symbol?
price_range = sqlContext.sql("SELECT _1 as symbol, MAX(_4) - MIN(_4) as price_range FROM stocks_table GROUP BY _1")
price_range.show()

symbol price_range         
AAPL   -0.25               
GOOGL  0.18000000000000682 
IBM    -0.14000000000000057
MSFT   -1.1299999999999955 
TSLA   -17.309999999999945 


In [37]:
# 9. What is the date on which each symbol experienced the highest price?
date_highest_price = sqlContext.sql("""
SELECT s._1 as symbol, s._2 as transaction_date, s._4 as highest_price
FROM stocks_table s
JOIN (
    SELECT _1 as symbol, MAX(_4) as max_price
    FROM stocks_table
    GROUP BY _1
) t
ON s._1 = t.symbol AND s._4 = t.max_price
""")
date_highest_price.show()

symbol transaction_date    highest_price
IBM    2009-03-27          99.8600      
TSLA   2022-01-28          987.6900     
MSFT   2018-05-25          98.9800      
GOOGL  2023-10-27 19:48:00 122.2800     
GOOGL  2023-10-27 19:46:00 122.2800     
GOOGL  2023-10-27 19:44:00 122.2800     
GOOGL  2023-10-27 19:43:00 122.2800     
GOOGL  2023-10-27 19:42:00 122.2800     
GOOGL  2023-10-27 19:40:00 122.2800     
GOOGL  2023-10-27 19:39:00 122.2800     
AAPL   2000-06-02          99.7500      


##3C

In [39]:
# import data from MYSQL database
blocks_df = sqlCtx.load(source="jdbc", 
                         url="jdbc:mysql://localhost/test?user=training&password=training", 
                         dbtable="blocks_2023_Sep_10_to_15")

blocks_info_df = sqlCtx.load(source="jdbc", 
                         url="jdbc:mysql://localhost/test?user=training&password=training", 
                         dbtable="blocks_info_2023_Sep_10_to_15")

tx_info_df = sqlCtx.load(source="jdbc", 
                         url="jdbc:mysql://localhost/test?user=training&password=training", 
                         dbtable="tx_info_2023_Sep_10_to_15")

In [40]:
# 1. How many total blocks are there in your blocks table?
total_blocks = blocks_df.count()
print("Total number of blocks:", total_blocks)

('Total number of blocks:', 919L)


In [41]:
# 2. What is the largest block height among the blocks in your blocks table?
largest_block_height = blocks_info_df.agg({"height": "max"}).collect()[0][0]
print("Largest block height:", largest_block_height)

('Largest block height:', 807290)


In [42]:
# 3. What is the date and time for that block?
largest_block_time = blocks_df.filter(blocks_df["block_index"] == largest_block_height).select("time").collect()[0][0]
print("Date and time for the largest block:", largest_block_time)

('Date and time for the largest block:', datetime.datetime(2023, 9, 12, 0, 0))


In [43]:
# 4. What is the largest number of transactions in your blocks?
largest_num_transactions = tx_info_df.groupBy("block_index").count().agg({"count": "max"}).collect()[0][0]
print("Largest number of transactions in a block:", largest_num_transactions)

('Largest number of transactions in a block:', 7252)
